In [1]:
import pandas as pd #Basic packages for creating dataframes and loading dataset
import numpy as np

import matplotlib.pyplot as plt #Package for visualization

import re #importing package for Regular expression operations

from sklearn.model_selection import train_test_split #Package for splitting the data

from sklearn.preprocessing import LabelEncoder #Package for conversion of categorical to Numerical

from keras.preprocessing.text import Tokenizer #Tokenization
from keras.preprocessing.sequence import pad_sequences #Add zeros or crop based on the length
from keras.models import Sequential #Sequential Neural Network
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D #For layers in Neural Network
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.
/Users/lavanya/opt/anaconda3/envs/env_full/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/lavanya/opt/anaconda3/envs/env_full/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/lavanya/opt/anaconda3/envs/env_full/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1

## Apply the code on spam data set available in the source code (text classification on the spam.csv data set)

In [2]:
data = pd.read_csv('spam.csv', encoding='latin-1') #Loading the dataset

In [3]:
data

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [4]:
data = data[['v1','v2']] # Keeping only the neccessary columns

In [5]:
data['v2'] = data['v2'].apply(lambda x: x.lower()) #converting to lower case
data['v2'] = data['v2'].apply((lambda x: re.sub('[^a-zA-Z0-9\s]', '', x))) #only a-z,A-Z,0-9 would be remaining in the data, else special characters are removed

/Users/lavanya/opt/anaconda3/envs/env_full/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/lavanya/opt/anaconda3/envs/env_full/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [6]:
for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ') #Removing Retweets

In [7]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['v2'].values)
X = tokenizer.texts_to_sequences(data['v2'].values)

In [8]:
X = pad_sequences(X)

embed_dim = 128
lstm_out = 196

In [9]:
def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(2,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model
# print(model.summary())

In [10]:
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['v1'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

In [11]:
batch_size = 32
model = createmodel()

%load_ext tensorboard

import tensorflow as tf
import datetime, os


logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
model.fit(X_train, Y_train, epochs =5, batch_size=batch_size, verbose = 2, validation_data=(X_test, Y_test),callbacks=[tensorboard_callback])
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)


Train on 3733 samples, validate on 1839 samples
Epoch 1/5
 - 62s - loss: 0.1696 - accuracy: 0.9421 - val_loss: 0.0814 - val_accuracy: 0.9734
Epoch 2/5
 - 63s - loss: 0.0421 - accuracy: 0.9869 - val_loss: 0.0654 - val_accuracy: 0.9821
Epoch 3/5
 - 67s - loss: 0.0212 - accuracy: 0.9930 - val_loss: 0.0680 - val_accuracy: 0.9837
Epoch 4/5
 - 67s - loss: 0.0110 - accuracy: 0.9973 - val_loss: 0.0786 - val_accuracy: 0.9826
Epoch 5/5
 - 67s - loss: 0.0063 - accuracy: 0.9979 - val_loss: 0.0824 - val_accuracy: 0.9821
0.08239430653629191
0.9820554852485657


In [12]:
print(model.metrics_names)

['loss', 'accuracy']


In [16]:
%tensorboard --logdir logs

In [15]:
!kill 6625